**Reference Link:** [RAG Systems Essentials (Analytics Vidhya)](https://courses.analyticsvidhya.com/courses/take/rag-systems-essentials/lessons/60148017-hands-on-deep-dive-into-rag-evaluation-metrics-generator-metrics-i)

In [2]:
%run Build_RAG_Pipeline_with_Source.ipynb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.11 requires langchain<0.4.0,>=0.3.11, but you have langchain 0.3.10 which is incompatible.
Metadata: {'id': 10, 'title': 'Artificial Intelligence'}
Content Brief:


Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.


Metadata: {'id': 3, 'title': 'Natural Language Processing (NLP)'}
Content Brief:


NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.


Metadata: {'id': 1, 'title': 'Machine Learning'}
Content Brief:


Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.


Metadata: {'id': 5, 'title': 'Photosynthesis'}
Content Brief:


Photosynthesis is the process plants use to convert sunlight into energy. This process produces glucose and releases oxygen as a byproduct. It is crucial for sustaining life on Earth by providing food and oxygen.

# Retriever Evaluation Metrics

![](https://i.imgur.com/5S4FhMB.png)

The retrieval process generally includes these steps:

- Convert the initial input query into an embedding using an embedding model of your choice (e.g., OpenAI's `text-embedding-3` model).
- Conduct a vector search with the embedded input on a vector database that holds your vectorized knowledge base, retrieving the top-K most "similar" document chunks.
- Optionally user a Reranker to rerank the retrieved results


Key Metrics to Evaluate here include:

- Contextual Precision
- Contextual Recall
- Contextual Relevancy

## Contextual Precision

The contextual precision metric measures your RAG pipeline's retriever by evaluating whether document chunks (nodes) in your `retrieval_context` that are relevant to the given `input` are ranked higher than irrelevant ones.

`deepeval`'s contextual precision metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a judge.

In `deepeval`, to use the ContextualPrecisionMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query
- `actual_output` : Actual LLM Response (not used in the computation)
- `expected_output` : Expected LLM Response (ground truth answer)
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/oVwrRAU.png)





In [3]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

{'context': [Document(metadata={'id': 10, 'title': 'Artificial Intelligence'}, page_content="Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning."),
  Document(metadata={'id': 3, 'title': 'Natural Language Processing (NLP)'}, page_content='NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.'),
  Document(metadata={'id': 1, 'title': 'Machine Learning'}, page_content='Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation 

### Example:

In [4]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

["Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.',
 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.']

In [5]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                  like virtual assistants, robotics and autonomous vehicles."""

In [6]:
new_context = ['Machine Learning is the study of algorithms which learn with more data',
               'AI is known as Artificial Intelligence'] + retrieved_context
new_context

['Machine Learning is the study of algorithms which learn with more data',
 'AI is known as Artificial Intelligence',
 "Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.',
 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.']

In [7]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualPrecisionMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metric = ContextualPrecisionMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

/Users/sourav.banerjee/Documents/My Codebases/GenerativAI_Demystified/venv/lib/python3.11/site-packages/deepeval/__init__.py:49: UserWarning: You are using deepeval version 1.4.7, however version 3.2.5 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:08,  8.89s/test case]

**************************************************
Contextual Precision Verbose Logs
**************************************************

Verdicts:
[
    {
        "verdict": "no",
        "reason": "This context is about machine learning, which is a subset of AI, but it doesn't directly define AI or mention its applications as required by the expected output."
    },
    {
        "verdict": "yes",
        "reason": "The context provides a direct definition of AI as 'Artificial Intelligence', aligning with the expected output's initial clarification."
    },
    {
        "verdict": "yes",
        "reason": "This context elaborates on what AI encompasses, including applications like 'virtual assistants, robotics, and autonomous vehicles', which matches the expected output."
    },
    {
        "verdict": "no",
        "reason": "While related to AI, this context specifically focuses on NLP, a branch of AI, without addressing the general concept or applications of AI as a whole."
    }

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [8]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

Sucess: True
Score: 0.5833333333333333
Reason: The score is 0.58 because the first node in the retrieval context is not directly about AI but rather about 'machine learning, which is a subset of AI, but it doesn't directly define AI or mention its applications as required by the expected output.' This irrelevant node is ranked higher than relevant nodes, such as the second node, which provides 'a direct definition of AI as 'Artificial Intelligence.'' Another relevant node is the third, which 'elaborates on what AI encompasses, including applications like 'virtual assistants, robotics, and autonomous vehicles.' Additionally, the fourth node focuses on 'NLP, a branch of AI, without addressing the general concept or applications of AI as a whole,' and the fifth focuses on 'machine learning, a subset of AI,' both of which are irrelevant yet ranked higher than some relevant nodes. These rankings lower the precision score.


## Contextual Recall

The contextual recall metric measures the quality of your RAG pipeline's retriever by evaluating the extent of which the `retrieval_context` aligns with the `expected_output`.

`deepeval`'s contextual recall metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the ContextualRecallMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query (not used in the computation)
- `actual_output` : Actual LLM Response (not used in the computation)
- `expected_output` : Expected LLM Response (ground truth answer)
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/PDbwuX5.png)





In [9]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

{'context': [Document(metadata={'id': 10, 'title': 'Artificial Intelligence'}, page_content="Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning."),
  Document(metadata={'id': 3, 'title': 'Natural Language Processing (NLP)'}, page_content='NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.'),
  Document(metadata={'id': 1, 'title': 'Machine Learning'}, page_content='Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation 

### Example 1:

In [10]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

["Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.',
 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.']

In [11]:
human_answer = """AI, also known as Artificial Intelligence is used to build complex systems for applications
                  like virtual assistants, robotics and autonomous vehicles."""

In [12]:
new_context = ['NVIDIA makes chips for AI', 'AI is an acronym for Artificial Intellence']
new_context

['NVIDIA makes chips for AI', 'AI is an acronym for Artificial Intellence']

In [13]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRecallMetric
from deepeval import evaluate

test_case1 = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=retrieved_context
)

test_case2 = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metric = ContextualRecallMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case1, test_case2], [metric])

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 2 test case(s) in parallel: |██████████|100% (2/2) [Time Taken: 00:02,  1.43s/test case]

**************************************************
Contextual Recall Verbose Logs
**************************************************

Verdicts:
[
    {
        "verdict": "yes",
        "reason": "The sentence can be attributed to the 2nd node in the retrieval context: 'AI is an acronym for Artificial Intellence'."
    }
]
 
Score: 1.0
Reason: The score is 1.00 because every sentence in the expected output is perfectly aligned with the nodes in the retrieval context. Great job!

**************************************************
Contextual Recall Verbose Logs
**************************************************

Verdicts:
[
    {
        "verdict": "yes",
        "reason": "The sentence can be attributed to the 1st node in the retrieval context, which states: 'AI includes applications like virtual assistants, robotics, and autonomous vehicles...'"
    }
]
 
Score: 1.0
Reason: The score is 1.00 because the expected output is perfectly aligned with the information from the node(s) in retri

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [14]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

Sucess: True
Score: 1.0
Reason: The score is 1.00 because every sentence in the expected output is perfectly aligned with the nodes in the retrieval context. Great job!


In [15]:
print('Sucess:', result.test_results[1].metrics_data[0].success)
print('Score:', result.test_results[1].metrics_data[0].score)
print('Reason:', result.test_results[1].metrics_data[0].reason)

Sucess: True
Score: 1.0
Reason: The score is 1.00 because the expected output is perfectly aligned with the information from the node(s) in retrieval context. Great job!


## Contextual Relevancy

The contextual relevancy metric measures the quality of your RAG pipeline's retriever by evaluating the overall relevance of the information presented in your `retrieval_context` for a given `input`.

`deepeval`'s contextual relevancy metric is a self-explaining LLM-Eval, meaning it outputs a reason for its metric score using an LLM as a Judge.

In `deepeval`, to use the ContextualRelevancyMetric, you'll have to provide the following arguments when creating an `LLMTestCase`:

- `input` : Input Query
- `actual_output` : Actual LLM Response (not used in the computation)
- `retrieval_context` : Top-N retrieved document chunks (nodes) from Vector DB


![](https://i.imgur.com/VLKoEsI.png)





In [16]:
query = "What is AI?"
response = rag_chain_w_sources.invoke(query)
response

{'context': [Document(metadata={'id': 10, 'title': 'Artificial Intelligence'}, page_content="Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning."),
  Document(metadata={'id': 3, 'title': 'Natural Language Processing (NLP)'}, page_content='NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.'),
  Document(metadata={'id': 1, 'title': 'Machine Learning'}, page_content='Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation 

### Example 1:

In [17]:
retrieved_context = [doc.page_content for doc in response['context']]
retrieved_context

["Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.',
 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.']

In [18]:
new_context = ['NVIDIA makes chips for AI', 'Google and Microsoft are battling out the market share for AI Chatbots'] + retrieved_context
new_context

['NVIDIA makes chips for AI',
 'Google and Microsoft are battling out the market share for AI Chatbots',
 "Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning. AI includes applications like virtual assistants, robotics, and autonomous vehicles. It's evolving rapidly with advancements in machine learning and deep learning.",
 'NLP is a branch of AI that enables computers to understand, interpret, and generate human language. Techniques include tokenization, stemming, and sentiment analysis. Applications range from chatbots to language translation services.',
 'Machine learning is a field of artificial intelligence focused on enabling systems to learn patterns from data. Algorithms analyze past data to make predictions or classify information. Popular applications include recommendation systems and image recognition.']

In [19]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import ContextualRelevancyMetric
from deepeval import evaluate

test_case = LLMTestCase(
    input=response['question'],
    actual_output=response['response'],
    expected_output=human_answer,
    retrieval_context=new_context
)

metric = ContextualRelevancyMetric(
    threshold=0.5,
    model="gpt-4o",
    include_reason=True,
    verbose_mode=True
)

result = evaluate([test_case], [metric])

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:04,  4.05s/test case]

**************************************************
Contextual Relevancy Verbose Logs
**************************************************

Verdicts:
[
    {
        "verdicts": [
            {
                "statement": "NVIDIA makes chips for AI",
                "verdict": "no",
                "reason": "The statement 'NVIDIA makes chips for AI' is about NVIDIA's products, not about what AI is."
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "Google and Microsoft are battling out the market share for AI Chatbots",
                "verdict": "no",
                "reason": "The statement 'Google and Microsoft are battling out the market share for AI Chatbots' is about competition between companies, not about what AI is."
            }
        ]
    },
    {
        "verdicts": [
            {
                "statement": "Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning.",

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

In [20]:
print('Sucess:', result.test_results[0].metrics_data[0].success)
print('Score:', result.test_results[0].metrics_data[0].score)
print('Reason:', result.test_results[0].metrics_data[0].reason)

Sucess: True
Score: 0.8181818181818182
Reason: The score is 0.82 because while some parts of the context focus on companies and market dynamics, the statements like 'Artificial intelligence refers to machines mimicking human intelligence, like problem-solving and learning.' and 'AI includes applications like virtual assistants, robotics, and autonomous vehicles.' directly address what AI is, providing substantial relevant information.
